## Import libraries

In [128]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
import numpy as np

## Import data

In [5]:
mean_interest = pd.read_csv("E:/Projects/real-estate-analysis/data/mean_interest_q.csv")
requests_vl = pd.read_csv("E:/Projects/real-estate-analysis/data/permits_reg.csv")
prices = pd.read_csv("E:/Projects/real-estate-analysis/data/annual_prices.csv")

In [18]:
prices["CD_TYPE_NL"].unique()

array(['Huizen met 2 of 3 gevels (gesloten + halfopen bebouwing)',
       'Huizen met 4 of meer gevels (open bebouwing)',
       'Alle huizen met 2, 3, 4 of meer gevels (excl. appartementen)',
       'Appartementen'], dtype=object)

In [29]:
prices[(prices["CD_TYPE_NL"] == "Appartementen") & 
       (prices["CD_PERIOD"].isin(["Q1", "Q2", "Q3", "Q4"])) &
       (prices["CD_REFNIS_NL"] == "VLAAMS GEWEST")].head()

,CD_YEAR,CD_TYPE_NL,CD_TYPE_FR,CD_REFNIS,CD_REFNIS_NL,CD_REFNIS_FR,CD_PERIOD,CD_CLASS_SURFACE,MS_TOTAL_TRANSACTIONS,MS_P_25,MS_P_50_median,MS_P_75,CD_niveau_refnis,year:period
13405,2010,Appartementen,Appartements,2000,VLAAMS GEWEST,REGION FLAMANDE,Q1,totaal / total,3623.0,110000.0,148000.0,192250.0,2,2010:Q1
13406,2010,Appartementen,Appartements,2000,VLAAMS GEWEST,REGION FLAMANDE,Q2,totaal / total,4152.0,115000.0,150000.0,195000.0,2,2010:Q2
13407,2010,Appartementen,Appartements,2000,VLAAMS GEWEST,REGION FLAMANDE,Q3,totaal / total,3897.0,116000.0,151480.0,195000.0,2,2010:Q3
13408,2010,Appartementen,Appartements,2000,VLAAMS GEWEST,REGION FLAMANDE,Q4,totaal / total,4150.0,115000.0,155000.0,205000.0,2,2010:Q4
31269,2011,Appartementen,Appartements,2000,VLAAMS GEWEST,REGION FLAMANDE,Q1,totaal / total,4044.0,115000.0,152900.0,200000.0,2,2011:Q1


In [72]:
model_data[model_data["year:period"] == "2012:Q2"].index[0]

9

In [90]:
def calc_price_lvl(current, prev):
    return (current / prev * 100 - 100)

In [98]:
# Create 1 dataframe
# Add data together
model_data = mean_interest[["year:period", "Value"]].merge(requests_vl,how="left", on="year:period")
prices_app = prices[(prices["CD_TYPE_NL"] == "Appartementen") & 
       (prices["CD_PERIOD"].isin(["Q1", "Q2", "Q3", "Q4"])) &
       (prices["CD_REFNIS_NL"] == "VLAAMS GEWEST")][["year:period", "MS_P_75", "MS_TOTAL_TRANSACTIONS"]].rename(columns={"MS_P_75":"Price_app", "MS_TOTAL_TRANSACTIONS":"App_transactions"})
model_data = model_data.merge(prices_app, on="year:period", how="left")

prices_house = prices[(prices["CD_TYPE_NL"] == "Huizen met 4 of meer gevels (open bebouwing)") & 
       (prices["CD_PERIOD"].isin(["Q1", "Q2", "Q3", "Q4"])) &
       (prices["CD_REFNIS_NL"] == "VLAAMS GEWEST")][["year:period", "MS_P_75", "MS_TOTAL_TRANSACTIONS"]].rename(columns={"MS_P_75":"Price_det", "MS_TOTAL_TRANSACTIONS":"Det_transactions"})
model_data = model_data.merge(prices_house, on="year:period", how="left")

prices_sem_det = prices[(prices["CD_TYPE_NL"] == "Huizen met 2 of 3 gevels (gesloten + halfopen bebouwing)") & 
       (prices["CD_PERIOD"].isin(["Q1", "Q2", "Q3", "Q4"])) &
       (prices["CD_REFNIS_NL"] == "VLAAMS GEWEST")][["year:period", "MS_P_75", "MS_TOTAL_TRANSACTIONS"]].rename(columns={"MS_P_75":"Price_sem_det", "MS_TOTAL_TRANSACTIONS":"Sem_det_transactions"})
model_data = model_data.merge(prices_sem_det, on="year:period", how="left")

# Add price level column
model_data["lvl_app"] = [None] + [calc_price_lvl(model_data["Price_app"].iloc[i], model_data["Price_app"].iloc[i-1]) for i in range(1, len(model_data)) ]
model_data["lvl_det"] = [None] + [calc_price_lvl(model_data["Price_det"].iloc[i], model_data["Price_det"].iloc[i-1]) for i in range(1, len(model_data)) ]
model_data["lvl_sem_det"] = [None] + [calc_price_lvl(model_data["Price_sem_det"].iloc[i], model_data["Price_sem_det"].iloc[i-1]) for i in range(1, len(model_data)) ]

model_data = model_data.dropna()
model_data.head()

,year:period,Value,Year,Refnis,Month,dwellings,appartments,one-dwelling buildings,surface area,Price_app,App_transactions,Price_det,Det_transactions,Price_sem_det,Sem_det_transactions,lvl_app,lvl_det,lvl_sem_det
8,2012:Q1,3.780000,2012.0,6000.0,6.0,5275.0,10115.0,5385.0,4730.0,220000.0,3826.0,375000.0,2831.0,250000.0,7087.0,2.325581,1.351351,2.040816
9,2012:Q2,3.330000,2012.0,6000.0,15.0,4375.0,8977.0,5134.0,3843.0,212500.0,4520.0,370000.0,3696.0,250000.0,8685.0,-3.409091,-1.333333,0.000000
10,2012:Q3,2.613333,2012.0,6000.0,24.0,3889.0,7546.0,4116.0,3430.0,216500.0,4297.0,375000.0,3667.0,255000.0,8438.0,1.882353,1.351351,2.000000
11,2012:Q4,2.276667,2012.0,6000.0,33.0,3674.0,7209.0,3990.0,3219.0,219000.0,4816.0,380000.0,3977.0,255000.0,8959.0,1.154734,1.333333,0.000000
12,2013:Q1,2.340000,2013.0,6000.0,6.0,4399.0,9120.0,5235.0,3885.0,219000.0,4243.0,375000.0,3366.0,252500.0,7813.0,0.000000,-1.315789,-0.980392


## Building Model
### Lasso regression

In [114]:
# Create training and test data for appartments
X_app = model_data[["Value", "appartments", "App_transactions"]]
y_app = model_data["lvl_app"]
X_train, X_test, y_train, y_test = train_test_split(X_app, y_app, test_size=0.2, random_state=0)

In [118]:
# Fit lasso regression model
app_model = Lasso(alpha=0.1, normalize=True)
app_model.fit(X_train, y_train)

Lasso(alpha=0.1, normalize=True)

In [119]:
# Check predictions
y_pred = app_model.predict(X_test)
pd.DataFrame(y_pred, y_test)

,0
lvl_app,
-0.387597,0.107902
2.127660,0.776927
-2.127660,0.953926
1.176471,0.702982
1.208333,0.828637
3.296703,0.806685
0.000000,0.847610
1.882353,0.709081


In [117]:
mean_squared_error(y_test, y_pred)

2.533373593554521

### KNN

In [137]:
app_model_knn = KNeighborsRegressor(2)
app_model_knn.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=2)

In [138]:
y_pred_knn = app_model.predict(X_test)
pd.DataFrame(y_pred_knn, y_test)

,0
lvl_app,
-0.387597,0.107902
2.127660,0.776927
-2.127660,0.953926
1.176471,0.702982
1.208333,0.828637
3.296703,0.806685
0.000000,0.847610
1.882353,0.709081
